In [43]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/pylab.py:159: UserWarning: pylab import has clobbered these variables: ['prod']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [44]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [45]:
import os

os.environ['MOSEKLM_LICENSE_FILE'] = '/content/mosek.lic'

In [46]:
# For Google Colab use, commands installing packages
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

# Install PICOS and CVXOPT in Google Colab
if IN_COLAB:
    !pip install -q picos
    !pip install -q mosek

In [47]:
import picos as pic
import numpy as np

In [48]:
print('Solvers supported on this installation of picos:', pic.solvers.all_solvers().keys())
print('Solvers available to picos on this machine :', pic.solvers.available_solvers())

Solvers supported on this installation of picos: dict_keys(['cplex', 'cvxopt', 'ecos', 'glpk', 'gurobi', 'mosek', 'mskfsn', 'osqp', 'scip', 'smcp'])
Solvers available to picos on this machine : ['cvxopt', 'ecos', 'mosek', 'mskfsn', 'osqp']


In [49]:
def P23_matrix(da, db1, db2):
    """
    Create the permutation matrix S_{b1,b2} for dimensions m, n, and p
    such that it permutes the tensor product as:
    S_{b1,b2}(|\psi_{a}⟩ ⊗ |\phi_{b_{1}}⟩ ⊗ |\theta_{b_{2}}⟩) = |\psi_{a}⟩ ⊗ (|\theta_{b_{1}}⟩ ⊗ |\phi_{b_{2}}⟩)

    Parameters:
    da (int): Dimension of |\psi_a⟩
    db1 (int): Dimension of |\phi_{b_1}⟩
    db2 (int): Dimension of |\theta_{b_2}⟩

    Returns:
    numpy.ndarray: The permutation matrix
    """
    # Total size of the tensor product space
    total_size = da * db1 * db2

    # Create the permutation matrix
    P = np.zeros((total_size, total_size))

    for a in range(da):
        for b1 in range(db1):
            for b2 in range(db2):
                old_index = a * db1 * db2 + b1 * db2 + b2
                new_index = a * db1 * db2 + b2 * db1 + b1
                P[new_index, old_index] = 1

    return P

In [50]:
# P23 Test
# Define two matrices
phi1 = np.array([[1],
              [2]])
da = 2

phi2 = np.array([[2],
              [3]])
db1 = 2

phi3 = np.array([[4],
              [5]])
db2 = 2

# create permatutation matrix
Sab1b2 = P23_matrix(da, db1, db2)

if np.all(Sab1b2 @ np.kron(np.kron(phi1, phi2), phi3)) == np.all(Sab1b2 @ np.kron(np.kron(phi1, phi3), phi2)):
  print("Permutation matrix is correct")
else:
  print("Permutation matrix is incorrect")

Permutation matrix is correct


# Example 1

Given a bell state $|\phi⟩ = \frac{1}{\sqrt{2}}(|00⟩ + |11⟩)$ check if $\rho_{ab}$ has a $k=1$ extension.

In [51]:
#State to check separability
phiMat = np.array([[1.,0.,0.,1.],
                [0.,0.,0.,0.],
                [0.,0.,0.,0.],
                [1.,0.,0.,1.]])/2

da = 2
db1 = 2
dB = db1
db = dB

In [52]:
#Constants
#----------
pab = pic.Constant("pab", phiMat)

Iab = pic.Constant('Iab', np.eye(da*db))

shpaB = (da*dB,da*dB)
shpab = (da*db, da*db)
shpsys = (da, db)

#Variables
#----------
W = pic.HermitianVariable('W', shpab)
Z0 = pic.HermitianVariable('Z0', shpab)
Z1 = pic.HermitianVariable('Z1', shpaB)

The $k=1$ formulation is
$$\max_{W, Z_{0}, Z_{1}}\rho_{ab} \boldsymbol{⋅} W$$
$$\text{subject to } W + Z_{0} + \mathcal{T}_{b}(Z_{1}) = 0$$
$$I_{ab} \boldsymbol{⋅} W = -1$$
$$Z_{0}, Z_{1} \succeq 0$$

In [53]:
prob1P = pic.Problem()

#Constraints
#----------
prob1P.add_constraint(W + Z0 + pic.partial_transpose(Z1, subsystems=(1), dimensions=shpsys) == 0)

prob1P.add_constraint(pic.trace(Iab * W) == -1)

prob1P.add_constraint(Z1 >> 0)

prob1P.add_constraint(Z0 >> 0)

<4×4 Complex LMI Constraint: Z0 ≽ 0>

In [54]:
#Objective
#----------
prob1P.set_objective('max',pic.trace(pab*W))

#User readable view of the problem being composed in PICOS'
print(prob1P)

Complex Semidefinite Program
  maximize tr(pab·W)
  over
    4×4 hermitian variables W, Z0, Z1
  subject to
    W + Z0 + Z1.{[2×2]⊗[2×2]ᵀ} = 0
    tr(Iab·W) = -1
    Z1 ≽ 0
    Z0 ≽ 0


In [55]:
#Solve the problem using cvxopt as a solver
prob1P.solve(verbosity=True,solver='mosek')

           PICOS 2.4.17            
Problem type: Complex Semidefinite Program.
Searching a solution strategy for MOSEK via Optimizer API.
Solution strategy:
  1. ExtraOptions
  2. ComplexAffineToRealReformulation
  3. ComplexLMIToRealReformulation
  4. MOSEKSolver
Applying ExtraOptions.
Applying ComplexAffineToRealReformulation.
Applying ComplexLMIToRealReformulation.
Building a MOSEK problem instance.
Starting solution search.
-----------------------------------
               MOSEK               
         via Optimizer API         
-----------------------------------
Problem
  Name                   :                 
  Objective sense        : maximize        
  Type                   : CONIC (conic optimization problem)
  Constraints            : 105             
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 48              
  Matrix variables       : 2 (scalarized: 72)

<primal feasible solution pair (claimed optimal) from mosek>

In [56]:
print('Status at the end of solving the problem usng mosek:', prob1P.status)

mu1P =  prob1P.value
print('Least mu obtained from solving the SDP using mosek above', mu1P)
print('Since this SDP value is positive, the state is entangled')

Status at the end of solving the problem usng mosek: optimal
Least mu obtained from solving the SDP using mosek above 0.49999999996591
Since this SDP value is positive, the state is entangled


# Example 2

Check for the existence of PPT symmetric extension with $k=1$ for a two qutrit state
$$p_{ab} = \frac{2}{7}|\psi_{+}⟩⟨\psi_{+}| + \frac{α}{7}\sigma_{+} + \frac{5 -  \alpha}{7}S_{ab}\sigma_{+} + S_{ab}$$
where $|\psi_{+}⟩ = \frac{1}{\sqrt{3}}(|00⟩ + |11⟩ + |22⟩)$, $|\sigma_{+}⟩ = \frac{1}{\sqrt{3}}(|01⟩⟨01| + |12⟩⟨12| + |20⟩⟨20|)$, $S_{ab}$ is the swap operator, and $0 \leq \alpha \leq \frac{5}{2}$

In [57]:
da = 3
db = 3
db1 = db
dB = db1
dab = da * db
daB = da * dB

#Construct input state
idMat = np.eye(da*db1)
psiP = idMat[0] + idMat[4] + idMat[8]
psiP = np.outer(psiP,psiP)
psiP = psiP/np.trace(psiP)

sigPlus = np.outer(idMat[1],idMat[1]) + np.outer(idMat[5],idMat[5]) + np.outer(idMat[6],idMat[6])
sigPlus = sigPlus/np.trace(sigPlus)
sigPlusEx = np.outer(idMat[3],idMat[3]) + np.outer(idMat[7],idMat[7]) + np.outer(idMat[2],idMat[2])
sigPlusEx = sigPlusEx/np.trace(sigPlusEx)

al = 0.75
rhoMat = (2/7)*psiP + (al/7)*sigPlus + ((5-al)/7)*sigPlusEx

In [58]:
#Constants
#----------
pab = pic.Constant("pab", rhoMat)
Iab = pic.Constant('Iab', np.eye(da*db))

shpab = (da*db, da*db)
shpaB = (da*dB,da*dB)
shpsys = (da, db)

#Variables
#----------
W = pic.HermitianVariable('W', shpab)
Z1 = pic.HermitianVariable('Z1', shpaB)
Z0 = pic.HermitianVariable('Z0', shpab)

The $k=1$ formulation is
$$\max_{W, Z_{0}, Z_{1}}\rho_{ab} \boldsymbol{⋅} W$$
$$\text{subject to } W + Z_{0} + \mathcal{T}_{b}(Z_{1}) = 0$$
$$I_{ab} \boldsymbol{⋅} W = -1$$
$$Z_{0}, Z_{1} \succeq 0$$

In [59]:
prob2P = pic.Problem()

#Constraints
#----------
prob2P.add_constraint(W + Z0 + pic.partial_transpose(Z1, subsystems=(1), dimensions=shpsys) == 0)

prob2P.add_constraint(pic.trace(Iab * W) == -1)

prob2P.add_constraint(Z1 >> 0)

prob2P.add_constraint(Z0 >> 0)

<9×9 Complex LMI Constraint: Z0 ≽ 0>

In [60]:
#Objective
#----------
prob2P.set_objective('max', pic.trace(pab * W))

#User readable view of the problem being composed in PICOS'
print(prob2P)

Complex Semidefinite Program
  maximize tr(pab·W)
  over
    9×9 hermitian variables W, Z0, Z1
  subject to
    W + Z0 + Z1.{[3×3]⊗[3×3]ᵀ} = 0
    tr(Iab·W) = -1
    Z1 ≽ 0
    Z0 ≽ 0


In [61]:
#Solve the problem using cvxopt as a solver
prob2P.solve(verbosity=True,solver='mosek')

           PICOS 2.4.17            
Problem type: Complex Semidefinite Program.
Searching a solution strategy for MOSEK via Optimizer API.
Solution strategy:
  1. ExtraOptions
  2. ComplexAffineToRealReformulation
  3. ComplexLMIToRealReformulation
  4. MOSEKSolver
Applying ExtraOptions.
Applying ComplexAffineToRealReformulation.
Applying ComplexLMIToRealReformulation.
Building a MOSEK problem instance.
Starting solution search.
-----------------------------------
               MOSEK               
         via Optimizer API         
-----------------------------------
Problem
  Name                   :                 
  Objective sense        : maximize        
  Type                   : CONIC (conic optimization problem)
  Constraints            : 505             
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 243             
  Matrix variables       : 2 (scalarized: 342

<primal feasible solution pair (claimed optimal) from mosek>

In [62]:
print('Status at the end of solving the problem using mosek:', prob2P.status)
mu2P =  prob2P.value

print('The input state parameter alpha is', al)
print('Least mu obtained from solving the SDP using mosek above is', mu2P)

Status at the end of solving the problem using mosek: optimal
The input state parameter alpha is 0.75
Least mu obtained from solving the SDP using mosek above is 0.007501735823247499


# Example 3

Given a bell state $|\phi⟩ = \frac{1}{\sqrt{2}}(|00⟩ + |11⟩)$ check if $\rho_{ab}$ has a $k=2$ extension.

In [63]:
#State to check separability
phiMat = np.array([[1.,0.,0.,1.],
                [0.,0.,0.,0.],
                [0.,0.,0.,0.],
                [1.,0.,0.,1.]])/2
da = 2
db = 2
db1 = db
db2 = db
dB = db1*db2

In [64]:
#Constants
#----------
pab = pic.Constant("pab", phiMat)


Ia = pic.Constant('Ia', np.eye(da))
Ib = pic.Constant('Ib', np.eye(db))
Iab = pic.Constant('Iab', np.eye(da*db))
IaB = pic.Constant('IaB', np.eye(da*dB))
Ib1b2 = pic.Constant('Ib1b2', np.eye(db1*db2))
Iab1b2 = pic.Constant('Iab1b2', np.eye(da*db1*db2))

Sb1b2 = pic.Constant('Sb1b2', P23_matrix(da, db1, db2)) # swap
prod = pic.Constant('IIb1b2', (Iab1b2 + Sb1b2)/2) # projector

shpa = (da, da)
shpab = (da*db,da*db)
shpb = (db,db)
shpB = (dB,dB)
shpaB = (da*dB, da*dB)
shpsys = (da,db1,db2)

#Variables
#----------
W = pic.HermitianVariable('W', shpab)
Z0 = pic.HermitianVariable('Z0', shpaB)
Z1 = pic.HermitianVariable('Z1', shpaB)
Z2 = pic.HermitianVariable('Z2', shpaB)

The $k=2$ formulation is
$$\max_{W, Z_{j}} \rho_{ab} \boldsymbol{\cdot} W$$
$$\text{ subject to }\prod_{b_{1}b_{2}}(W \otimes I_{b_{2}} + Z_{0} + \mathcal{T}_{b_{1}}(Z_{1}) + {T}_{b_{1},b_{2}}(Z_{2}))\prod_{b_{1}b_{2}} = 0$$
$$I_{ab} \boldsymbol{\cdot} W = -1$$
$$Z_{0}, Z_{1}, Z_{2} \succeq 0$$

In [65]:
prob3P = pic.Problem()

#Constraints
#----------
prob3P.add_constraint(prod*(W @ Ib + Z0 + pic.partial_transpose(Z1, subsystems=(1), dimensions=shpsys) + pic.partial_transpose(Z2, subsystems=(1, 2), dimensions=shpsys))*prod == 0)

prob3P.add_constraint(pic.trace(Iab * W) == -1)

prob3P.add_constraint(Z0 >> 0)
prob3P.add_constraint(Z1 >> 0)
prob3P.add_constraint(Z2 >> 0)

<8×8 Complex LMI Constraint: Z2 ≽ 0>

In [66]:
#Objective
#----------
prob3P.set_objective('max', pic.trace(pab*W))

#User readable view of the problem being composed in PICOS'
print(prob3P)

Complex Semidefinite Program
  maximize tr(pab·W)
  over
    4×4 hermitian variable W
    8×8 hermitian variable Zi ∀ i ∈ [0…2]
  subject to
    IIb1b2·(W⊗Ib + Z0 + Z1.{[2×2]⊗[2×2]ᵀ⊗[2×2]} +
      Z2.{[2×2]⊗[2×2]ᵀ⊗[2×2]ᵀ})·IIb1b2 = 0
    tr(Iab·W) = -1
    Z0 ≽ 0
    Z1 ≽ 0
    Z2 ≽ 0


In [67]:
#Solve the problem using cvxopt as a solver
prob3P.solve(verbosity=True,solver='mosek')

           PICOS 2.4.17            
Problem type: Complex Semidefinite Program.
Searching a solution strategy for MOSEK via Optimizer API.
Solution strategy:
  1. ExtraOptions
  2. ComplexAffineToRealReformulation
  3. ComplexLMIToRealReformulation
  4. MOSEKSolver
Applying ExtraOptions.
Applying ComplexAffineToRealReformulation.
Applying ComplexLMIToRealReformulation.
Building a MOSEK problem instance.
Starting solution search.
-----------------------------------
               MOSEK               
         via Optimizer API         
-----------------------------------
Problem
  Name                   :                 
  Objective sense        : maximize        
  Type                   : CONIC (conic optimization problem)
  Constraints            : 537             
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 208             
  Matrix variables       : 3 (scalarized: 408

<primal feasible solution pair (claimed optimal) from mosek>

In [68]:
print('Status at the end of solving the problem:', prob3P.status)

mu3P =  prob3P.value

print('Least mu obtained from solving the SDP above is', mu3P)

Status at the end of solving the problem: optimal
Least mu obtained from solving the SDP above is 0.4999999897329403


# Example 4

Check for the existence of PPT symmetric extension with $k=2$ for a two qutrit state
$$p_{ab} = \frac{2}{7}|\psi_{+}⟩⟨\psi_{+}| + \frac{α}{7}\sigma_{+} + \frac{5 -  \alpha}{7}S_{ab}\sigma_{+} + S_{ab}$$
where $|\psi_{+}⟩ = \frac{1}{\sqrt{3}}(|00⟩ + |11⟩ + |22⟩)$, $|\sigma_{+}⟩ = \frac{1}{\sqrt{3}}(|01⟩⟨01| + |12⟩⟨12| + |20⟩⟨20|)$, $S_{ab}$ is the swap operator, and $0 \leq \alpha \leq \frac{5}{2}$

In [69]:
da = 3
db = 3
db1 = db
db2 = db
dB = db1*db2

In [70]:
#Re-construct input state from Example 2
idMat = np.eye(da*db)
psiP = idMat[0] + idMat[4] + idMat[8]
psiP = np.outer(psiP,psiP)
psiP = psiP/np.trace(psiP)

In [71]:
sigPlus = np.outer(idMat[1],idMat[1]) + np.outer(idMat[5],idMat[5]) + np.outer(idMat[6],idMat[6])
sigPlus = sigPlus/np.trace(sigPlus)
sigPlusEx = np.outer(idMat[3],idMat[3]) + np.outer(idMat[7],idMat[7]) + np.outer(idMat[2],idMat[2])
sigPlusEx = sigPlusEx/np.trace(sigPlusEx)

In [72]:
alpha = 1.95
rhoMat = (2/7)*psiP + (al/7)*sigPlus + ((5-al)/7)*sigPlusEx

In [73]:
#Constants
#----------
pab = pic.Constant("pab", rhoMat)
pa = pic.partial_trace(pab, subsystems=(1), dimensions=(db, db))

Ia = pic.Constant('Ia', np.eye(da))
Ib = pic.Constant('Ib', np.eye(db))
Iab = pic.Constant('Iab', np.eye(da*db))
IaB = pic.Constant('IaB', np.eye(da*dB))
Ib1b2 = pic.Constant('Ib1b2', np.eye(db1*db2))
Iab1b2 = pic.Constant('Iab1b2', np.eye(da*db1*db2))

Sb1b2 = pic.Constant('Sb1b2', P23_matrix(da, db1, db2))
prod = pic.Constant('IIb1b2', (Iab1b2 + Sb1b2)/2)

shpab = (da*db, da*db)
shpB = (dB, dB)
shpaB = (da*dB, da*dB)
shpsys = (da,db1,db2)

#Variables
#----------
W = pic.HermitianVariable('W', shpab)
Z0 = pic.HermitianVariable('Z0', shpaB)
Z1 = pic.HermitianVariable('Z1', shpaB)
Z2 = pic.HermitianVariable('Z2', shpaB)

The $k=2$ formulation is
$$\max_{W, Z_{j}} \rho_{ab} \boldsymbol{\cdot} W$$
$$\text{ subject to }\prod_{b_{1}b_{2}}(W \otimes I_{b_{2}} + Z_{0} + \mathcal{T}_{b_{1}}(Z_{1}) + {T}_{b_{1},b_{2}}(Z_{2}))\prod_{b_{1}b_{2}} = 0$$
$$I_{ab} \boldsymbol{\cdot} W = -1$$
$$Z_{0}, Z_{1}, Z_{2} \succeq 0$$

In [74]:
prob4P = pic.Problem()

#Constraints
#----------
prob4P.add_constraint(prod*(W @ Ib + Z0 + pic.partial_transpose(Z1, subsystems=(1), dimensions=shpsys) + pic.partial_transpose(Z2, subsystems=(1, 2), dimensions=shpsys))*prod == 0)

prob4P.add_constraint(pic.trace(Iab * W) == -1)

prob4P.add_constraint(Z0 >> 0)
prob4P.add_constraint(Z1 >> 0)
prob4P.add_constraint(Z2 >> 0)

<27×27 Complex LMI Constraint: Z2 ≽ 0>

In [75]:
#Objective
#----------
prob4P.set_objective('max', pic.trace(pab*W))

#User readable view of the problem being composed in PICOS'
print(prob4P)

Complex Semidefinite Program
  maximize tr(pab·W)
  over
    9×9 hermitian variable W
    27×27 hermitian variable Zi ∀ i ∈ [0…2]
  subject to
    IIb1b2·(W⊗Ib + Z0 + Z1.{[3×3]⊗[3×3]ᵀ⊗[3×3]} +
      Z2.{[3×3]⊗[3×3]ᵀ⊗[3×3]ᵀ})·IIb1b2 = 0
    tr(Iab·W) = -1
    Z0 ≽ 0
    Z1 ≽ 0
    Z2 ≽ 0


In [76]:
#Solve the problem using cvxopt as a solver
prob4P.solve(verbosity=True,solver='mosek')

           PICOS 2.4.17            
Problem type: Complex Semidefinite Program.
Searching a solution strategy for MOSEK via Optimizer API.
Solution strategy:
  1. ExtraOptions
  2. ComplexAffineToRealReformulation
  3. ComplexLMIToRealReformulation
  4. MOSEKSolver
Applying ExtraOptions.
Applying ComplexAffineToRealReformulation.
Applying ComplexLMIToRealReformulation.
Building a MOSEK problem instance.
Starting solution search.
-----------------------------------
               MOSEK               
         via Optimizer API         
-----------------------------------
Problem
  Name                   :                 
  Objective sense        : maximize        
  Type                   : CONIC (conic optimization problem)
  Constraints            : 5914            
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 2268            
  Matrix variables       : 3 (scalarized: 445

<primal feasible solution pair (claimed optimal) from mosek>

In [77]:
print('Status at the end of solving the problem using mosek:', prob4P.status)
mu4P =  prob4P.value

print('The input state parameter alpha is', alpha)
print('Least mu obtained from solving the SDP using mosek above is', mu4P)

Status at the end of solving the problem using mosek: optimal
The input state parameter alpha is 1.95
Least mu obtained from solving the SDP using mosek above is 0.03288970375681763
